Pre-[processing]

In [ ]:
import pandas as pd
import glob
import re

# 1. Load & Clean ISPU (Semua Tahun)
def clean_all_ispu(path_pattern):
    # Menggunakan glob untuk list semua file csv
    all_files = glob.glob(path_pattern)
    
    # Cek apakah file ditemukan
    if not all_files:
        print(f"Peringatan: Tidak ada file yang ditemukan di path: {path_pattern}")
        return None

    all_dfs = []
    for filename in all_files:
        df = pd.read_csv(filename)
        # Tambahkan logika cleaning Master di sini jika ada
        all_dfs.append(df)
    
    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    return None

# 2. Load Data Cuaca
def load_weather():
    weather_files = {
        'DKI1': 'datasets/cuaca-harian/cuaca-harian-dki1-bundaranhi.csv',
        'DKI2': 'datasets/cuaca-harian/cuaca-harian-dki2-kelapagading.csv',
        'DKI3': 'datasets/cuaca-harian/cuaca-harian-dki3-jagakarsa.csv',
        'DKI4': 'datasets/cuaca-harian/cuaca-harian-dki4-lubangbuaya.csv',
        'DKI5': 'datasets/cuaca-harian/cuaca-harian-dki5-kebonjeruk.csv'
    }
    w_dfs = []
    for sid, path in weather_files.items():
        df = pd.read_csv(path)
        df['time'] = pd.to_datetime(df['time'])
        df['stasiun_id'] = sid
        w_dfs.append(df)
    return pd.concat(w_dfs, ignore_index=True)

# --- EKSEKUSI ---
df_ispu = clean_all_ispu('datasets/ispu/*.csv')
df_weather = load_weather()

# Gabungkan ISPU + Cuaca
final_df = pd.merge(df_ispu, df_weather, 
                    left_on=['tanggal', 'stasiun_id'], 
                    right_on=['time', 'stasiun_id'], 
                    how='left').sort_values(['stasiun_id', 'tanggal'])

# 3. Fitur Tambahan (Feature Engineering)
final_df['month'] = final_df['tanggal'].dt.month
final_df['day_of_week'] = final_df['tanggal'].dt.dayofweek

# Tambah Fitur Lag1 (Nilai PM10/PM2.5 kemarin)
for col in ['pm_sepuluh', 'pm_duakomalima']:
    final_df[f'{col}_lag1'] = final_df.groupby('stasiun_id')[col].shift(1)

# Hapus kolom tanggal yang double dan simpan
final_df.drop(columns=['time'], inplace=True)
final_df.to_csv('processed_data_ready.csv', index=False)

print("Master, data sudah rapi dan tersimpan di 'processed_data_ready.csv'!")

ValueError: No objects to concatenate